In [50]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.cluster import MiniBatchKMeans,KMeans
import time

In [51]:
#Reading the image and converting it into RGB values -
image=cv2.imread('../input/testimage/test.jpg')

plt.figure()
plt.axis('off')
plt.imshow(image)

image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
plt.figure()
plt.axis('off')
plt.imshow(image)

In [52]:
h,w,c=image.shape
image.shape

In [53]:
def init_kmeans(feats,k):
    return KMeans(n_clusters=k).fit(feats)

def kmeans_sklearn(image_2d,K):
    
    #clustering pixels in appropriate clusters
    sse=[]
    K=range(1,K)
    for k in K:
        kmeansModel=init_kmeans(image_2d,k)
        sse.append(kmeansModel.inertia_)
        
    #Finding the best k
    plt.plot(K,sse,'bx-')
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.title('The elbow method showing the optiomal')
    plt.show()
    
def assign_color(image,k):
    
    #assigning colors of the nearest centroids to pixels
    kmeansModel=init_kmeans(image,k)
    cluster=kmeansModel.labels_
    centroids=kmeansModel.cluster_centers_
    cluster_color = image.copy()
    for i in range(k):
        mask = cluster == i
        cluster_color[mask] = centroids[i]
        
    # 3. Visualize the result
    plt.imshow(np.reshape(cluster_color, (h,w,c)))
        
def compare_performance(image,k):
    
    k_means=KMeans(init='k-means++',n_clusters=k,n_init=10)
    t0=time.time()
    k_means.fit(image)
    t_batch=time.time()-t0
    print('Time taken for k_means:',t_batch)
    print('Inertia of k_means:',k_means.inertia_)
    
    mini_batch = MiniBatchKMeans(init='k-means++',n_clusters=k,batch_size=50,n_init=10)
    t0=time.time()
    mini_batch.fit(image)
    t_mini_batch=time.time()-t0
    print('Time taken for mini batch kmeans:',t_mini_batch)
    print('Inertia of mini batch:',mini_batch.inertia_)

In [54]:
#Reshape image
image_2d=np.reshape(image,(h*w,c))

#finding cluster and optimal k
kmeans_sklearn(image_2d,11)

#From below curve we see 4 to be the optimal k value
best_k=4

#Assign centroid color to the cluster
assign_color(image_2d,best_k)

#compare the performance
compare_performance(image_2d,best_k)